# L-Layer Testing
## Architecture

```json
{
    "epochs": 1,
    "layers": 2,
    "activations": {
        "layer1": "sigmoid",
        "layer2": "relu"
    },
    "neurons": {
        "layer1": 3,
        "layer2": 1
    },
    "weight": 0.01,
    "bias": 0,
    "learning_rate": 0.005
}
```

---

---
## Forward Propogation
### S3 Event

In [1]:
context = ''
# Simulate S3 event trigger data
event = {
    "Records": [
        {
            "eventVersion": "2.0",
            "eventTime": "1970-01-01T00:00:00.000Z",
            "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
             },
            "s3": {
                "configurationId": "testConfigRule",
                "object": {
                    "eTag": "0123456789abcdef0123456789abcdef",
                    "sequencer": "0A1B2C3D4E5F678901",
                    "key": "training_input/datasets.h5",
                    "size": 1024
                },
                "bucket": {
                    "arn": "arn:aws:s3:::lnn",
                    "name": "lnn",
                    "ownerIdentity": {
                        "principalId": "EXAMPLE"
                    }
                },
                "s3SchemaVersion": "1.0"
            },
            "responseElements": {
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH",
                "x-amz-request-id": "EXAMPLE123456789"
            },
            "awsRegion": "us-west-2",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "EXAMPLE"
            },
            "eventSource": "aws:s3"
        }
    ]
}

---
### Launch

In [2]:
import launch
from launch import *
launch.lambda_handler(event, context)
np.random.seed(1)

Complete Neural Network Settings: 

{
    "activations": {
        "layer1": "relu",
        "layer2": "sigmoid"
    },
    "bias": 0,
    "data_keys": {
        "W1": "W1|float64#3#12288",
        "W2": "W2|float64#1#3",
        "b1": "b1|float64#3#1",
        "b2": "b2|float64#1#1",
        "grads": "grads|json",
        "m": "m|int",
        "results": "results|json",
        "test_set_x": "test_set_x|float64#12288#50",
        "test_set_y": "test_set_y|int64#1#50",
        "train_set_x": "train_set_x|float64#12288#209",
        "train_set_y": "train_set_y|int64#1#209"
    },
    "dims": {
        "test_set_x": [
            12288,
            50
        ],
        "test_set_y": [
            1,
            50
        ],
        "train_set_x": [
            12288,
            209
        ],
        "train_set_y": [
            1,
            209
        ]
    },
    "epochs": 1,
    "input_data": [
        "train_set_x",
        "train_set_y",
        "test_set_x",
        "test_set

---
### Launch -> Trainer

In [3]:
event = {
    "parameter_key": "parameters|json",
    "state": "start"
}

In [4]:
import trainer
from trainer import *
trainer.lambda_handler(event, context)

Starting Forward Propogation for epoch 0, layer 1
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 1,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 2,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "relu",
    "epoch": 0,
    "id": 3,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
### Trainer -> Neuron (Layer 1)

In [5]:
import neuron
from neuron import *

event = {
    "1": {
        "activation": "relu",
        "epoch": 0,
        "id": 1,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "2": {
        "activation": "relu",
        "epoch": 0,
        "id": 2,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "3": {
        "activation": "relu",
        "epoch": 0,
        "id": 3,
        "last": "True",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    }
}
event = dumps(event)
event = loads(event)

In [6]:
layer = 3
parameters = from_cache(endpoint=endpoint, key="parameters|json")
if layer == parameters['layers'] + 1:
    print("True")

True


In [7]:
parameter_key = event.get('1')['parameter_key']
parameters = from_cache(endpoint=endpoint, key=parameter_key)
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'grads': 'grads|json',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 1,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [8]:
layer = event.get('1')['layer']
print("Processing Layer: " + str(layer))
event1 = event.get('1')
print("Neuron 1 event:\n" + dumps(event1))
neuron.lambda_handler(event=event1, context='')
event2 = event.get('2')
print("Neuron 2 event:\n" + dumps(event2))
neuron.lambda_handler(event=event2, context='')
event3 = event.get('3')
neuron.lambda_handler(event=event3, context='')
print("Neuron 3 event:\n" + dumps(event3))

Processing Layer: 1
Neuron 1 event:
{"activation": "relu", "epoch": 0, "id": 1, "last": "False", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}
Neuron 2 event:
{"activation": "relu", "epoch": 0, "id": 2, "last": "False", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}
Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Neuron 3 event:
{"activation": "relu", "epoch": 0, "id": 3, "last": "True", "layer": 1, "parameter_key": "parameters|json", "state": "forward"}


---
### Neuron (Layer 1) -> Trainer

In [9]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

Propogating forward onto Layer 2
Starting Forward Propogation for epoch 0, layer 2
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


**Sanity Check: Confirm that `TrainerLambda` processed Layer 1 Activations**

In [10]:
parameters = from_cache(endpoint=endpoint, key='parameters|json')
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'grads': 'grads|json',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [11]:
A1 = from_cache(endpoint=endpoint, key=parameters['data_keys']['A1'])
A1.shape

(3, 209)

---
### Trainer -> Neuron (Layer 2)

In [12]:
import neuron
from neuron import *
event = {
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron.lambda_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
### Neuron (Layer 2) -> Trainer

In [13]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

Cost after epoch 0: 0.6933895808784364
Starting Backward Propogation for epoch 0, layer 2
Payload to be sent to NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "backward"
}


**Sanity Check: Confirm `results` have been updated with the correct cost.**

In [14]:
parameters = from_cache(endpoint=endpoint, key="parameters|json")
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'A2': 'A2|float64#1#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'dZ2': 'dZ2|float64#1#209',
  'grads': 'grads|json',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

In [15]:
A2 = from_cache(endpoint=endpoint, key=parameters['data_keys']['A2'])
A2.shape

(1, 209)

In [16]:
tmp = from_cache(endpoint=endpoint, key=parameters['data_keys']['results'])
print("Current results output:\n")
tmp

Current results output:



{'epoch0': {'cost': 0.6933895808784364}}

---

---
## Back Propogation
### Trainer -> Neuron (Layer 2)
** Sanity Check: Confirm that the derivative of the Cost Function (`dA2`) is initialized.**

In [17]:
parameters = from_cache(endpoint=endpoint, key="parameters|json")
parameters

{'activations': {'layer1': 'relu', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_keys': {'A1': 'A1|float64#3#209',
  'A2': 'A2|float64#1#209',
  'W1': 'W1|float64#3#12288',
  'W2': 'W2|float64#1#3',
  'b1': 'b1|float64#3#1',
  'b2': 'b2|float64#1#1',
  'dZ2': 'dZ2|float64#1#209',
  'grads': 'grads|json',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209'},
 'dims': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0.01}

>**Note:** The derivative of the Cost is already built within the `TrainerLambda` and the event already sent to the `NeuronLambda`. Therefore the next step in the workflow is to process the derivatives for Layer 2 on the Neuron.

---
### Neuron (Layer 2) -> Trainer

In [18]:
def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [19]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [25]:
# set the scene
layer = 2
ID = 1
# get dZ2
dz_name = 'dZ' + str(layer)
print(dz_name)
dz = from_cache(endpoint=endpoint,key=parameters['data_keys'][dz_name])
dz.shape

dZ2


(1, 209)

In [33]:
#get activations for the previous layer
a_name = 'A' + str(layer-1)
A_prev = from_cache(
    endpoint=endpoint,
    key=parameters['data_keys'][a_name]
)
A_prev.shape

(3, 209)

>**Note:** The shape is based on the previous layer's neuron size, so how to we handle this?

In [66]:
layer_config=[784, 100, 100, 10]

In [67]:
num_layers = len(layer_config)

In [68]:
for i in range(num_layers-1):
    print(i)

0
1
2


In [69]:
layer_config[-1]

10

In [70]:
for i in range(num_layers-2, 0, -1):
    print(i)

2
1


**Hmmmmmm I think I'm working on the wrong layer**